# Inference

This script showcases the different models available in fishbAIT and how to use them efficiently.

In [1]:
import torch
import pandas as pd
import numpy as np
from inference_utils.fishbAIT_for_inference import fishbAIT_for_inference

Specify the model version and load the model

In [2]:
MODEL_TYPE = 'EC10'
SPECIES_GROUP = 'invertebrates'
MODEL_VERSION = f'{MODEL_TYPE}_{SPECIES_GROUP}'

In [3]:
fishbait = fishbAIT_for_inference(model_version=MODEL_VERSION)
fishbait.load_fine_tuned_model()

Load the SMILES you wish to predict

In [4]:
data = pd.read_excel('../data/tutorials/Inference_example_2.xlsx')
data

,SMILES,cmpdname
0,CC(=O)Oc1ccccc1C(O)=O,Aspirin
1,[Cr],Chromium
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,Fluoxetine hydrochloride
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,Clofenotane
4,[Cu],Copper
...,...,...
995,[Pb++].[O-]c1c(cc(c([O-])c1[N+]([O-])=O)[N+]([...,Lead styphnate
996,CC(C)(C)C(O)(CCc1ccc(Cl)cc1)Cn2cncn2,Tebuconazole
997,[Na+].[Na+].[Na+].[Na+].OCCN(CCO)c1nc(Nc2ccc(c...,OpticalBrightenerBbu220
998,CNC.OC(=O)COc1ccc(Cl)cc1Cl,"2,4-D dimethylamine salt"


Specify the endpoint and effect you wish to predict and make the prediction

In [5]:
PREDICTION_ENDPOINT = 'EC10'
PREDICTION_EFFECT = 'MOR'
EXPOSURE_DURATION = 96
SMILES_COLUMN_NAME = 'SMILES'

In [6]:
results = fishbait.predict_toxicity(SMILES = data[SMILES_COLUMN_NAME].iloc[0:10].tolist(), exposure_duration=EXPOSURE_DURATION, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, return_cls_embeddings=True)
results

Renamed NOEC *EC10* in 0 positions
Did not return onehotencoding for Endpoint. Why? You specified only one Endpoint or you specified NOEC and EC10 which are coded to be the same endpoint.


100%|██████████| 2/2 [00:00<00:00,  9.04it/s]


,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L),CLS_embeddings
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC10,MOR,CC(=O)Oc1ccccc1C(=O)O,1.480453,30.231068,"[-0.10645921528339386, -0.23677757382392883, 0..."
1,[Cr],1.982271,EC10,MOR,[Cr],0.858568,7.220513,"[0.18673408031463623, 0.037091806530952454, 0...."
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC10,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-1.714382,0.019303,"[-0.512883186340332, 0.2940675914287567, -0.53..."
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC10,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-2.274864,0.005311,"[-0.5919798016548157, 0.1586819887161255, 0.27..."
4,[Cu],1.982271,EC10,MOR,[Cu],-1.465667,0.034224,"[-0.3312265872955322, 0.5778264403343201, -0.6..."
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC10,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,-0.014564,0.967020,"[0.3484106659889221, -0.051787130534648895, -0..."
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC10,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,1.257319,18.085024,"[-0.351306676864624, -0.6592566967010498, 0.22..."
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC10,MOR,CC(Br)(CO)[N+](=O)[O-],0.207973,1.614258,"[-0.5035646557807922, 0.06318626552820206, 0.1..."
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC10,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-1.510213,0.030888,"[-0.34705662727355957, 0.47176435589790344, -0..."
9,[Cl-].[Cl-].[Zn++],1.982271,EC10,MOR,[Cl-].[Cl-].[Zn+2],-0.339971,0.457118,"[0.01175069808959961, -0.2064513862133026, -0...."


If you run into truble using this model use the `__help__` function

In [7]:
fishbait.__help__()


        This is a python class used to load and use the fine-tuned deep-learning model `fishbAIT` for environmental toxicity predictions in fish algae and aquatic invertebrates.
        The models have been trained on a large corpus of SMILES (chemical representations) on data collected from various sources.

        Currently there are nine models available for use. The models are divided by toxicity endpoint and by species group. The models are the following:
        **Fish**
        - `EC50_fish` The EC50 model is trained on EC50 mortality (MOR) data and is thus suitable for the prediction of said endpoints.
        - `EC10_fish` The EC10 model is trained on EC10/NOEC data with various effects (mortality, intoxication, development, reproduction, morphology, growth and population) ab. (MOR, ITX, DVP, REP, MPH, GRO, POP)
        - `EC50EC10_fish` The EC50EC10 model is trained on EC50, EC10 and NOEC data with various effects (mortality, intoxication, development, reproduction, morphol

# Check the predictions compared to our training sets

In [8]:
from inference_utils.plots_for_space import PlotPCA_CLSProjection
from inference_utils.pytorch_data_utils import check_closest_chemical, check_training_data

Check if chemicals are present in training data. They may be present as either an:
- 'endpoint_match' i.e. the chemical was used for training this model for this species and endpoint.
- 'effect_match' i.e. the chemical was used for training this model for this species, endpoint and effect.

In [9]:
results = check_training_data(results, model_type=MODEL_TYPE, species_group=SPECIES_GROUP, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT)
results

,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L),CLS_embeddings,endpoint match,effect match
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC10,MOR,CC(=O)Oc1ccccc1C(=O)O,1.480453,30.231068,"[-0.10645921528339386, -0.23677757382392883, 0...",1,1
1,[Cr],1.982271,EC10,MOR,[Cr],0.858568,7.220513,"[0.18673408031463623, 0.037091806530952454, 0....",1,1
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC10,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-1.714382,0.019303,"[-0.512883186340332, 0.2940675914287567, -0.53...",1,1
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC10,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-2.274864,0.005311,"[-0.5919798016548157, 0.1586819887161255, 0.27...",1,1
4,[Cu],1.982271,EC10,MOR,[Cu],-1.465667,0.034224,"[-0.3312265872955322, 0.5778264403343201, -0.6...",1,1
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC10,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,-0.014564,0.967020,"[0.3484106659889221, -0.051787130534648895, -0...",1,1
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC10,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,1.257319,18.085024,"[-0.351306676864624, -0.6592566967010498, 0.22...",1,1
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC10,MOR,CC(Br)(CO)[N+](=O)[O-],0.207973,1.614258,"[-0.5035646557807922, 0.06318626552820206, 0.1...",0,0
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC10,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-1.510213,0.030888,"[-0.34705662727355957, 0.47176435589790344, -0...",1,1
9,[Cl-].[Cl-].[Zn++],1.982271,EC10,MOR,[Cl-].[Cl-].[Zn+2],-0.339971,0.457118,"[0.01175069808959961, -0.2064513862133026, -0....",1,1


Next we check if which chemical is closest to the predicted chemicals by evaluating the CLS-embeddings against the training set's CLS-embedding by means of their cosine-similarity:
- cosine-similarity=1 --> Identical structures
- cosine-similarity=-1 --> completely oposite in terms of toxicity

In [10]:
results = check_closest_chemical(results=results, MODELTYPE=MODEL_TYPE, PREDICTION_SPECIES=SPECIES_GROUP, PREDICTION_ENDPOINT=PREDICTION_ENDPOINT, PREDICTION_EFFECT=PREDICTION_EFFECT)
results

,SMILES,exposure_duration,endpoint,effect,SMILES_Canonical_RDKit,predictions log10(mg/L),predictions (mg/L),CLS_embeddings,endpoint match,effect match,most similar chemical,cosine similarity
0,CC(=O)Oc1ccccc1C(O)=O,1.982271,EC10,MOR,CC(=O)Oc1ccccc1C(=O)O,1.480453,30.231068,"[-0.10645921528339386, -0.23677757382392883, 0...",1,1,CC(=O)Oc1ccccc1C(=O)O,1.000000
1,[Cr],1.982271,EC10,MOR,[Cr],0.858568,7.220513,"[0.18673408031463623, 0.037091806530952454, 0....",1,1,[Cr],1.000000
2,[H+].[Cl-].CNCCC(Oc1ccc(cc1)C(F)(F)F)c2ccccc2,1.982271,EC10,MOR,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],-1.714382,0.019303,"[-0.512883186340332, 0.2940675914287567, -0.53...",1,1,CNCCC(Oc1ccc(C(F)(F)F)cc1)c1ccccc1.[Cl-].[H+],1.000000
3,Clc1ccc(cc1)C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl,1.982271,EC10,MOR,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,-2.274864,0.005311,"[-0.5919798016548157, 0.1586819887161255, 0.27...",1,1,Clc1ccc(C(c2ccc(Cl)cc2)C(Cl)(Cl)Cl)cc1,1.000000
4,[Cu],1.982271,EC10,MOR,[Cu],-1.465667,0.034224,"[-0.3312265872955322, 0.5778264403343201, -0.6...",1,1,[Cu],1.000000
5,CCNc1nc(Cl)nc(NC(C)C)n1,1.982271,EC10,MOR,CCNc1nc(Cl)nc(NC(C)C)n1,-0.014564,0.967020,"[0.3484106659889221, -0.051787130534648895, -0...",1,1,CCNc1nc(Cl)nc(NC(C)C)n1,1.000000
6,CN(C)C1=NC(=O)N(C2CCCCC2)C(=O)N1C,1.982271,EC10,MOR,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,1.257319,18.085024,"[-0.351306676864624, -0.6592566967010498, 0.22...",1,1,CN(C)c1nc(=O)n(C2CCCCC2)c(=O)n1C,1.000000
7,CC(Br)(CO)[N+]([O-])=O,1.982271,EC10,MOR,CC(Br)(CO)[N+](=O)[O-],0.207973,1.614258,"[-0.5035646557807922, 0.06318626552820206, 0.1...",0,0,O=[N+]([O-])C(Br)(CO)CO,0.908265
8,c1ccc2c(c1)c3cccc4cccc2c34,1.982271,EC10,MOR,c1ccc2c(c1)-c1cccc3cccc-2c13,-1.510213,0.030888,"[-0.34705662727355957, 0.47176435589790344, -0...",1,1,c1ccc2c(c1)-c1cccc3cccc-2c13,1.000000
9,[Cl-].[Cl-].[Zn++],1.982271,EC10,MOR,[Cl-].[Cl-].[Zn+2],-0.339971,0.457118,"[0.01175069808959961, -0.2064513862133026, -0....",1,1,[Cl-].[Cl-].[Zn+2],1.000000


# Plot PCA for all chemicals

Finally we can plot the chemical space built during training of the Transformer module in the model. The space is built by the CLS-embedidngs present in the training set of the model but can be used to project new chemicals onto. The space prepared in this example uses `show_all_predictions=True` which plots additional SMILES, not included in the training data to add interpretability. We also use `inference_df=results` to plot the predicted SMILES from above into the space, however this can be set to `None` if not desired.

The plot can be saved as interactive HTML by `fig.write_html(figurename.html')`

Note that in the hover text of each point, the L1Error from our 10x10-fold cross-validation is included from when the used model was evaluated on that chemical.

In [11]:
from inference_utils.plots_for_space import PlotPCA_CLSProjection

In [12]:
PlotPCA_CLSProjection(model_type=MODEL_TYPE, endpoint=PREDICTION_ENDPOINT, effect=PREDICTION_EFFECT, species_group=SPECIES_GROUP, show_all_predictions=True, inference_df=results)

c:\Users\skall\OneDrive - Chalmers\Documents\git_repo\tutorials\inference_utils\plots_for_space.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_preds['pc1'], all_preds['pc2'] = pcac[:len(all_preds),0], pcac[:len(all_preds),1]
c:\Users\skall\OneDrive - Chalmers\Documents\git_repo\tutorials\inference_utils\plots_for_space.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_preds['pc1'], all_preds['pc2'] = pcac[:len(all_preds),0], pcac[:len(all_preds),1]
